# 신용카드 크롤링 데이터 저장

In [ ]:
import requests
import json
import time
from bs4 import BeautifulSoup

# 최신 브라우저 헤더 설정
headers = {
    "authority": "api.card-gorilla.com:8080",
    "accept": "application/json, text/plain, */*",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_rag_merged_data(card_data):
    """동일 카드 내 중복 카테고리를 병합하여 RAG용 리스트 반환"""
    card_idx = card_data.get("idx")
    card_name = card_data.get("name")
    corp_name = card_data.get("corp_txt")
    
    detail_url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    try:
        res = requests.get(detail_url, headers=headers, timeout=10)
        res_data = res.json()
        
        key_benefits = res_data.get('key_benefit', [])
        merged_benefits = {}
        
        for kb in key_benefits:
            category = kb.get('title')
            html_content = kb.get('info', '')
            
            if html_content:
                soup = BeautifulSoup(html_content, 'html.parser')
                clean_text = soup.get_text(separator=" ", strip=True)
                
                if category not in merged_benefits:
                    merged_benefits[category] = []
                
                if clean_text not in merged_benefits[category]:
                    merged_benefits[category].append(clean_text)
        
        results = []
        for category, texts in merged_benefits.items():
            combined_text = " / ".join(texts)
            full_content = f"[{card_name}] {category} 혜택: {combined_text}"
            
            results.append({
                "content": full_content, # 벡터 DB 임베딩 및 검색용
                "metadata": {            # 필터링용 메타데이터
                    "card_id": card_idx,
                    "card_name": card_name,
                    "corp": corp_name,
                    "category": category,
                    "min_performance": card_data.get("pre_month_money"),
                    "annual_fee": card_data.get("annual_fee_basic")
                }
            })
        return results
    except Exception as e:
        print(f"   ⚠️ Error (ID {card_idx}): {e}")
        return []

# --- 전체 카드 전수 수집 시작 ---
all_rag_data = []
all_card_indices = []
page = 1

print("🚀 모든 카드의 데이터 수집을 시작합니다. (전체 페이지 순회)")

while True:
    list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate=CRD"
    try:
        response = requests.get(list_url, headers=headers, timeout=10)
        data = response.json()
        goods_list = data.get("data", [])

        # 더 이상 수집할 카드가 없으면 종료
        if not goods_list:
            print(f"\n✅ 수집을 완료했습니다. (마지막 페이지: {page-1})")
            break

        print(f"\n📄 현재 {page}페이지 수집 중... (현재 누적 데이터: {len(all_rag_data)}개)")

        for card in goods_list:
            c_name = card.get('name')
            c_idx = card.get('idx')
            
            print(f"  📦 {c_name} (ID: {c_idx}) 처리 중...")
            
            # 1. idx 리스트 추가
            all_card_indices.append(c_idx)
            
            # 2. 혜택 병합 및 수집
            benefit_units = get_rag_merged_data(card)
            all_rag_data.extend(benefit_units)
            
            # 서버 부하 방지용 짧은 휴식
            time.sleep(0.4)

        page += 1 # 다음 페이지로

    except Exception as e:
        print(f"❌ {page}페이지 호출 중 오류 발생: {e}")
        break

# --- 최종 결과 저장 ---

# 1. RAG용 상세 데이터 저장 (JSON)
with open("all_card_benefits_rag.json", "w", encoding="utf-8") as f:
    json.dump(all_rag_data, f, ensure_ascii=False, indent=4)

# 2. 마스터 idx 리스트 저장 (JSON)
with open("all_card_idx_list.json", "w", encoding="utf-8") as f:
    json.dump(all_card_indices, f, ensure_ascii=False, indent=4)

print("\n" + "="*50)
print(f"✨ 최종 수집 완료!")
print(f"📂 상세 혜택 데이터: {len(all_rag_data)}개 ('all_credit_benefits_rag.json')")
print(f"📂 수집된 카드 ID: {len(all_card_indices)}개 ('all_credit_idx_list.json')")
print("="*50)

🚀 모든 카드의 데이터 수집을 시작합니다. (전체 페이지 순회)

📄 현재 1페이지 수집 중... (현재 누적 데이터: 0개)
  📦 신한카드 Mr.Life (ID: 13) 처리 중...
  📦 삼성 iD SELECT ALL 카드 (ID: 2885) 처리 중...
  📦 카드의정석 SHOPPING+ (ID: 2687) 처리 중...
  📦 삼성카드 & MILEAGE PLATINUM (스카이패스) (ID: 49) 처리 중...
  📦 KB국민 My WE:SH 카드 (ID: 2441) 처리 중...
  📦 삼성카드 taptap O (ID: 51) 처리 중...
  📦 쿠팡 와우카드 (ID: 2609) 처리 중...
  📦 American Express® Gold Card Edition2 (ID: 2663) 처리 중...
  📦 삼성 iD SELECT ON 카드 (ID: 2886) 처리 중...
  📦 삼성 iD SIMPLE 카드 (ID: 2376) 처리 중...
  📦 신한카드 Discount Plan+ (ID: 2835) 처리 중...
  📦 신한카드 처음(ANNIVERSE) (ID: 2759) 처리 중...
  📦 LOCA 365 카드 (ID: 2330) 처리 중...
  📦 taptap DIGITAL (ID: 657) 처리 중...
  📦 LOCA LIKIT 1.2 (ID: 2261) 처리 중...
  📦 올바른 FLEX 카드 (ID: 666) 처리 중...
  📦 카드의정석 EVERY DISCOUNT (ID: 2719) 처리 중...
  📦 카드의정석2 (ID: 2848) 처리 중...
  📦 신한카드 Deep Oil (ID: 39) 처리 중...
  📦 THE iD. 1st (ID: 2915) 처리 중...

📄 현재 2페이지 수집 중... (현재 누적 데이터: 121개)
  📦 삼성 iD GLOBAL 카드 (ID: 2676) 처리 중...
  📦 THE 1 (스카이패스) (ID: 1909) 처리 중...
  📦 우리카드 MILE&POINT (ID: 28

# 체크카드 크롤링 저장

In [27]:
import requests
import json
import time
from bs4 import BeautifulSoup

# 최신 브라우저 헤더 설정
headers = {
    "authority": "api.card-gorilla.com:8080",
    "accept": "application/json, text/plain, */*",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_rag_merged_data(card_data):
    """동일 카드 내 중복 카테고리를 병합하여 RAG용 리스트 반환"""
    card_idx = card_data.get("idx")
    card_name = card_data.get("name")
    corp_name = card_data.get("corp_txt")
    
    # 체크카드의 상세 정보 API 주소 (동일함)
    detail_url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    try:
        res = requests.get(detail_url, headers=headers, timeout=10)
        res_data = res.json()
        
        key_benefits = res_data.get('key_benefit', [])
        merged_benefits = {}
        
        for kb in key_benefits:
            category = kb.get('title')
            html_content = kb.get('info', '')
            
            if html_content:
                soup = BeautifulSoup(html_content, 'html.parser')
                clean_text = soup.get_text(separator=" ", strip=True)
                
                if category not in merged_benefits:
                    merged_benefits[category] = []
                
                if clean_text not in merged_benefits[category]:
                    merged_benefits[category].append(clean_text)
        
        results = []
        for category, texts in merged_benefits.items():
            combined_text = " / ".join(texts)
            full_content = f"[{card_name}] {category} 혜택: {combined_text}"
            
            results.append({
                "content": full_content,
                "metadata": {
                    "card_id": card_idx,
                    "card_name": card_name,
                    "corp": corp_name,
                    "category": category,
                    "min_performance": card_data.get("pre_month_money"),
                    "annual_fee": card_data.get("annual_fee_basic"),
                    "card_type": "check" # 체크카드임을 명시
                }
            })
        return results
    except Exception as e:
        print(f"   ⚠️ Error (ID {card_idx}): {e}")
        return []

# --- 체크카드 전수 수집 시작 ---
all_check_data = []
all_check_indices = []
page = 1

print("🚀 모든 체크카드의 데이터 수집을 시작합니다. (cate=CHK)")

while True:
    # 카테고리를 CHK로 변경
    list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate=CHK"
    try:
        response = requests.get(list_url, headers=headers, timeout=10)
        data = response.json()
        goods_list = data.get("data", [])

        if not goods_list:
            print(f"\n✅ 수집 완료! (마지막 페이지: {page-1})")
            break

        print(f"\n📄 체크카드 {page}페이지 수집 중... (현재 누적: {len(all_check_data)}개)")

        for card in goods_list:
            c_name = card.get('name')
            c_idx = card.get('idx')
            
            print(f"  📦 {c_name} (ID: {c_idx}) 처리 중...")
            
            all_check_indices.append(c_idx)
            benefit_units = get_rag_merged_data(card)
            all_check_data.extend(benefit_units)
            
            time.sleep(0.4) # 서버 부하 방지

        page += 1

    except Exception as e:
        print(f"❌ {page}페이지 호출 중 오류 발생: {e}")
        break

# --- 최종 결과 저장 (파일명 구분) ---

with open("all_check_benefits_rag.json", "w", encoding="utf-8") as f:
    json.dump(all_check_data, f, ensure_ascii=False, indent=4)

with open("all_check_idx_list.json", "w", encoding="utf-8") as f:
    json.dump(all_check_indices, f, ensure_ascii=False, indent=4)

print("\n" + "="*50)
print(f"✨ 체크카드 수집 완료!")
print(f"📂 상세 혜택 데이터: {len(all_check_data)}개 ('all_check_benefits_rag.json')")
print(f"📂 수집된 카드 ID: {len(all_check_indices)}개 ('all_check_idx_list.json')")
print("="*50)

🚀 모든 체크카드의 데이터 수집을 시작합니다. (cate=CHK)

📄 체크카드 1페이지 수집 중... (현재 누적: 0개)
  📦 ONE 체크카드 (ID: 2749) 처리 중...
  📦 KB Youth Club 체크카드 (ID: 2929) 처리 중...
  📦 신한카드 SOL트래블 체크 (ID: 2667) 처리 중...
  📦 노리2 체크카드(KB Pay) (ID: 2422) 처리 중...
  📦 네이버페이 머니카드 (ID: 2626) 처리 중...
  📦 하나 나라사랑카드 (ID: 2934) 처리 중...
  📦 토스뱅크 체크카드 (ID: 2269) 처리 중...
  📦 트래블러스 체크카드(토심이) (ID: 2691) 처리 중...
  📦 KB 틴업 체크카드 (ID: 2852) 처리 중...
  📦 더나은 체크카드 (ID: 2601) 처리 중...
  📦 네이버페이 머니 하나 체크카드 (ID: 2432) 처리 중...
  📦 개이득 체크카드 (ID: 2519) 처리 중...
  📦 토심이 첵첵 체크카드 (ID: 2604) 처리 중...
  📦 신한카드 나라사랑카드 체크 (ID: 2933) 처리 중...
  📦 신한카드 Deep Dream 체크 (ID: 281) 처리 중...
  📦 카카오뱅크 프렌즈 체크카드 (ID: 435) 처리 중...
  📦 노리2 체크카드(Global) (ID: 2423) 처리 중...
  📦 카드의정석 오하CHECK (ID: 2570) 처리 중...
  📦 모빌리언스카드 (ID: 2321) 처리 중...
  📦 트래블로그 체크카드 (ID: 2394) 처리 중...

📄 체크카드 2페이지 수집 중... (현재 누적: 130개)
  📦 K-패스카드(체크) (ID: 2561) 처리 중...
  📦 노리체크카드 (ID: 348) 처리 중...
  📦 직장인보너스체크카드 (ID: 350) 처리 중...
  📦 트래블러스 체크카드 (ID: 2905) 처리 중...
  📦 NH20해봄체크카드 (ID: 362) 처리 중...
  📦 신한카드 O

In [28]:
import json
import pandas as pd

def check_json_quality(file_path):
    print(f"--- [{file_path}] 데이터 품질 체크 시작 ---")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 1. 데이터가 비어있는지 확인
    if not data:
        print("❌ 오류: 파일이 비어 있습니다.")
        return

    # Pandas 데이터프레임으로 변환 (분석 용이)
    df = pd.json_normalize(data)
    
    # 2. 필수 필드 누락 체크
    # content(혜택 내용), metadata.card_name, metadata.category 등
    required_columns = ['content', 'metadata.card_id', 'metadata.card_name', 'metadata.category']
    
    print(f"📊 총 데이터 개수: {len(df)}개")
    
    for col in required_columns:
        if col in df.columns:
            missing_count = df[col].isna().sum() or (df[col] == "").sum()
            if missing_count > 0:
                print(f"⚠️ 경고: {col} 필드에 {missing_count}개의 누락 데이터가 있습니다.")
            else:
                print(f"✅ {col}: 누락 없음")
        else:
            print(f"❌ 오류: 필수 필드 '{col}'가 데이터에 존재하지 않습니다.")

    # 3. 비정상적으로 짧은 혜택 내용 체크 (크롤링 오류 가능성)
    short_content = df[df['content'].str.len() < 20]
    if not short_content.empty:
        print(f"⚠️ 경고: 내용이 너무 짧은 데이터가 {len(short_content)}개 발견되었습니다. (예: {short_content['content'].iloc[0]})")

    # 4. 카드사별 데이터 분포 확인 (특정 카드사가 누락되었는지 확인용)
    print("\n🏢 카드사별 수집 현황 (상위 5개):")
    print(df['metadata.corp'].value_counts().head())
    
    print("-" * 50)

# 실행 (파일명에 맞춰 수정하세요)
check_json_quality("all_credit_benefits_rag.json")
check_json_quality("all_check_benefits_rag.json")

--- [all_credit_benefits_rag.json] 데이터 품질 체크 시작 ---
📊 총 데이터 개수: 5418개
✅ content: 누락 없음
✅ metadata.card_id: 누락 없음
✅ metadata.card_name: 누락 없음
✅ metadata.category: 누락 없음
⚠️ 경고: 내용이 너무 짧은 데이터가 2개 발견되었습니다. (예: [넥슨 현대카드] 선택형 혜택: )

🏢 카드사별 수집 현황 (상위 5개):
metadata.corp
KB국민카드    683
신한카드      649
우리카드      599
삼성카드      568
현대카드      540
Name: count, dtype: int64
--------------------------------------------------
--- [all_check_benefits_rag.json] 데이터 품질 체크 시작 ---
📊 총 데이터 개수: 2345개
✅ content: 누락 없음
✅ metadata.card_id: 누락 없음
✅ metadata.card_name: 누락 없음
✅ metadata.category: 누락 없음

🏢 카드사별 수집 현황 (상위 5개):
metadata.corp
KB국민카드     485
신한카드       334
우리카드       253
MG새마을금고    158
NH농협카드     154
Name: count, dtype: int64
--------------------------------------------------
